# Final Report 

#### Group Members: Ivan, Ruohan, Hiroshi, Samin

# (1) Introduction

# (2) Methods and Results

## a) “Exploratory Data Analysis (EDA)”

## b) “Methods: Plan”

# (3) Discussion

# (4) References